# Define ROIs
Overall rule: 
for regions that can be mapped with prf for each subject (V1/V2/V3/hV4), we will use those out of the box	    
V1d,V1v,V2d,V2v,V3d,V3v - dorsal and ventral subdivisions of V1, V2, and V3	    
hV4 - human V4	    

for regions that can be identified in the localizer, we will use probablisitic masks (so that we can localize the different patches, e.g., OPA vs. PPA) + t-value > 0 in a constrast

OFA - occipital face area	    
FFA-1 - posterior section of fusiform face area	    
FFA-2 - anterior section of fusiform face area	    
mTL-faces - face-selective region in middle portion of temporal lobe	    
aTL-faces - face-selective region in anterior portion of temporal lobe	    
OVWFA - occipital visual word form area	    
VWFA-1 - posterior section of visual word form area	    
VWFA-2 - anterior section of visual word form area	    
mfs-words - word-selective region located near the mid-fusiform sulcus	    
mTL-words - word-selective region in middle portion of temporal lobe	    
OPA - occipital place area	    
PPA - parahippocampal place area	    
RSC - retrospenial cortex (place-selective)	    
EBA - extrastriate body area (can also be referred to as LOTC-bodies (lateral occipitotemporal cortex))	    
FBA-1 - posterior section of fusiform body area (can also be referred to as VOTC-bodies-1 (ventral occipitotemporal cortex))	    
FBA-2 - anterior section of fusiform body area (can also be referred to as VOTC-bodies-2)	    
mTL-bodies - body-selective region in middle portion of temporal lobe	    

for others, we will use a combination of probablistic masks (e.g., Kastner 2015) and visual responsiveness (R-squared > 1% from an ON-OFF GLM). 	    
% 08 - VO1	    
% 09 - VO2	    
% 10 - PHC1	    
% 11 - PHC2	    
% 12 - MST	    
% 13 - hMT	   
% 14 - LO2	    
% 15 - LO1	   
% 16 - V3b	    
% 17 - V3a	  
% 18 - IPS0	   
% 19 - IPS1	    
% 20 - IPS2	    
% 21 - IPS3	  
% 22 - IPS4	  
% 23 - IPS5	  
% 24 - SPL1	

In [1]:
import os, sys, glob, pickle, configparser
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
exp_config = configparser.ConfigParser()
exp_config.read('../config')


# Set the directories
NSD_top_dir = exp_config['DIR']['NSD_top_dir']
FS_top_dir = os.path.join(NSD_top_dir, 'data', 'nsddata', 'freesurfer')


In [2]:
def extract_prf_roi(sub, roi_ind, hemi, fs_top_dir):
    prf = nib.load(os.path.join(fs_top_dir,
                                 'subj0{}'.format(sub),
                                 'label',
                                 '{}.prf-visualrois.mgz'.format(hemi))).get_fdata().squeeze()
    return prf==roi_ind

def extract_floc_roi(sub, roi_name, domain_name, hemi, fs_top_dir, prob_roi_dir, vis_resp=True):
    curr_sub_probROIs_dir = os.path.join(prob_roi_dir, 'subj0{}'.format(sub))
    curr_mask_dir = os.path.join(curr_sub_probROIs_dir,
                                    '{}.probmap_{}.mgz'.format(hemi, roi_name)
                                    )
    curr_mask = nib.load(curr_mask_dir).get_fdata().squeeze().astype(bool)

    curr_floc_dir = os.path.join(fs_top_dir,
                                 'subj0{}'.format(sub),
                             'label',
                             '{}.floc-{}.mgz'.format(hemi, domain_name))
    curr_floc_mask = nib.load(curr_floc_dir).get_fdata().squeeze().astype(bool)
    
    if vis_resp:
        return (curr_mask>0) & (curr_floc_mask)
    else:
        return curr_mask>0

def extract_Kaster_roi(sub, roi_ind, hemi, fs_top_dir, vis_resp=True):
    curr_R2_dir = os.path.join(fs_top_dir,
                              'subj0{}'.format(sub),
                            'label',
                            '{}.R2.mgz'.format(hemi))
    curr_R2 = nib.load(curr_R2_dir).get_fdata().squeeze()
    
    kastner_dir = os.path.join(fs_top_dir,
                              'subj0{}'.format(sub),
                            'label',
                            '{}.Kastner2015.mgz'.format(hemi))
    kastner = nib.load(kastner_dir).get_fdata().squeeze()
    
    if vis_resp:
        return (kastner==roi_ind) & (curr_R2 > 1)
    else:
        return kastner==roi_ind

In [3]:
sub_list = range(1, 9)
hemi_list = ['lh', 'rh']
prf_roi_list = list(range(1,8))
prf_roi_name_list = ['V1d','V1v','V2d','V2v','V3d','V3v','hV4']
floc_rois = ['OFA', 'FFA-1', 'FFA-2', 'mTL-faces', 'aTL-faces',# Face-selective
'OWFA', 'VWFA-1', 'VWFA-2', 'mfs-words', 'mTL-words', # Word-selective
'OPA', 'PPA', 'RSC', # Scene-selective
'EBA', 'FBA-1', 'FBA-2', 'mTL-bodies']# Body-selective
domain_list = ['faces' for k in range(5)] + ['words' for k in range(5)] + ['places' for k in range(3)] + ['bodies' for k in range(4)]
probROIs_dir = os.path.join(NSD_top_dir, 'intermediate', 'probROIs')
kastner_rois = list(range(8,25))
kastner_roi_names = ['VO1', 'VO2', 'PHC1', 'PHC2', 'MST','hMT','LO2', 'LO1', 'V3b', 'V3a', 'IPS0', 'IPS1',
                     'IPS2', 'IPS3', 'IPS4', 'IPS5', 'SPL1']
size_summary = pd.DataFrame(columns=['SUB','ROI', 'hemi', 'size', 'responsive only?'])

for curr_sub in sub_list:
    sub_roi_dict = {}
    for curr_hemi in hemi_list:
    # PRF-mapped ROIs
        for curr_roi_ind in prf_roi_list:
            sub_roi_dict['{}.{}'.format(curr_hemi,
                                   prf_roi_name_list[curr_roi_ind-1])]=extract_prf_roi(curr_sub,
                                                                                      curr_roi_ind,
                                                                                      curr_hemi,
                                                                                      FS_top_dir)
        # Functional localizer ROIs
        for curr_roi_ind, curr_roi in enumerate(floc_rois):
             sub_roi_dict['{}.{}'.format(curr_hemi,curr_roi)] = extract_floc_roi(curr_sub, 
                                                                                 curr_roi, 
                                                                                 domain_list[curr_roi_ind], 
                                                                                 curr_hemi, 
                                                                                 FS_top_dir,
                                                                                 probROIs_dir, vis_resp=False)    
        # Kastner 2015 ROIs
        for curr_roi_ind, curr_roi_code in enumerate(kastner_rois):
            sub_roi_dict['{}.{}'.format(curr_hemi,kastner_roi_names[curr_roi_ind])] = extract_Kaster_roi(curr_sub,
                                                                                                        curr_roi_code,
                                                                                                        curr_hemi,
                                                                                                        FS_top_dir,
                                                                                                        vis_resp=False)
    
    for curr_key in sub_roi_dict.keys():
        size_summary.loc[len(size_summary)] = {'SUB':curr_sub,
                                                'ROI':curr_key.split('.')[1], 
                                               'hemi':curr_key.split('.')[0], 
                                               'size':np.sum(sub_roi_dict[curr_key]),
                                              'responsive only?':'No'}

In [4]:
for curr_sub in sub_list:
    sub_roi_dict = {}
    for curr_hemi in hemi_list:
    # PRF-mapped ROIs
        for curr_roi_ind in prf_roi_list:
            sub_roi_dict['{}.{}'.format(curr_hemi,
                                   prf_roi_name_list[curr_roi_ind-1])]=extract_prf_roi(curr_sub,
                                                                                      curr_roi_ind,
                                                                                      curr_hemi,
                                                                                      FS_top_dir)
        # Functional localizer ROIs
        for curr_roi_ind, curr_roi in enumerate(floc_rois):
             sub_roi_dict['{}.{}'.format(curr_hemi,curr_roi)] = extract_floc_roi(curr_sub, 
                                                                                 curr_roi, 
                                                                                 domain_list[curr_roi_ind], 
                                                                                 curr_hemi, 
                                                                                 FS_top_dir,
                                                                                 probROIs_dir, vis_resp=True)    
        # Kastner 2015 ROIs
        for curr_roi_ind, curr_roi_code in enumerate(kastner_rois):
            sub_roi_dict['{}.{}'.format(curr_hemi,kastner_roi_names[curr_roi_ind])] = extract_Kaster_roi(curr_sub,
                                                                                                        curr_roi_code,
                                                                                                        curr_hemi,
                                                                                                        FS_top_dir,
                                                                                                        vis_resp=True)
    
    for curr_key in sub_roi_dict.keys():
        size_summary.loc[len(size_summary)] = {'SUB':curr_sub,
                                                'ROI':curr_key.split('.')[1], 
                                               'hemi':curr_key.split('.')[0], 
                                               'size':np.sum(sub_roi_dict[curr_key]),
                                              'responsive only?':'Yes'}

In [5]:
small_ROIs_df = size_summary[(size_summary['responsive only?']=='Yes')&
            (size_summary['size']<100)].sort_values('size')

In [6]:
small_ROIs_df['comb_name'] = small_ROIs_df['hemi']+ '.' + small_ROIs_df['ROI']
size_summary['comb_name'] = size_summary['hemi'] + '.' + size_summary['ROI']

In [14]:
size_summary[size_summary['ROI']=='EBA']

SUB  ROI hemi   size responsive only? comb_name
20     1  EBA   lh  14890               No    lh.EBA
61     1  EBA   rh  15110               No    rh.EBA
102    2  EBA   lh  15808               No    lh.EBA
143    2  EBA   rh  13377               No    rh.EBA
184    3  EBA   lh  16350               No    lh.EBA
225    3  EBA   rh  17011               No    rh.EBA
266    4  EBA   lh  15766               No    lh.EBA
307    4  EBA   rh  15344               No    rh.EBA
348    5  EBA   lh  13005               No    lh.EBA
389    5  EBA   rh  13830               No    rh.EBA
430    6  EBA   lh  16777               No    lh.EBA
471    6  EBA   rh  20526               No    rh.EBA
512    7  EBA   lh  12163               No    lh.EBA
553    7  EBA   rh  14781               No    rh.EBA
594    8  EBA   lh  15707               No    lh.EBA
635    8  EBA   rh  16759               No    rh.EBA
676    1  EBA   lh   4843              Yes    lh.EBA
717    1  EBA   rh   7733              Yes    rh.EBA
758    2  EBA   lh   7076              Yes    lh.EBA
799    2  EBA   rh   8692              Yes    rh.EBA
840    3  EBA   lh   7733              Yes    lh.EBA
881    3  EBA   rh   8757              Yes    rh.EBA
922    4  EBA   lh   7574              Yes    lh.EBA
963    4  EBA   rh   8525              Yes    rh.EBA
1004   5  EBA   lh   9879              Yes    lh.EBA
1045   5  EBA   rh  10698              Yes    rh.EBA
1086   6  EBA   lh   7675              Yes    lh.EBA
1127   6  EBA   rh  10842              Yes    rh.EBA
1168   7  EBA   lh   7381              Yes    lh.EBA
1209   7  EBA   rh   7051              Yes    rh.EBA
1250   8  EBA   lh   6747              Yes    lh.EBA
1291   8  EBA   rh   7453              Yes    rh.EBA

In [7]:
all_rois = size_summary['comb_name'].unique()
# Exclude any roi that has fewer than 100 visually responsive vertices in any subject
exclude_rois = small_ROIs_df['comb_name'].unique()
included_rois = [roi for roi in all_rois if roi not in exclude_rois]

In [23]:
size_summary_grouped_min = size_summary[(size_summary['responsive only?']=='Yes')&
                                   (size_summary['comb_name'].isin(included_rois))].groupby(['comb_name'])['size'].min().reset_index()
size_summary_grouped_max = size_summary[(size_summary['responsive only?']=='Yes')&
                                   (size_summary['comb_name'].isin(included_rois))].groupby(['comb_name'])['size'].max().reset_index()
size_summary_grouped_median = size_summary[(size_summary['responsive only?']=='Yes')&
                                   (size_summary['comb_name'].isin(included_rois))].groupby(['comb_name'])['size'].median().reset_index()

In [26]:
size_summary_grouped = size_summary_grouped_min.merge(size_summary_grouped_max, on = 'comb_name').merge(size_summary_grouped_median, on = 'comb_name')

In [29]:
size_summary_grouped.to_csv(os.path.join(NSD_top_dir,
                                         'results',
                                         'Visual_ROI_sizes.csv'))

In [30]:
size_summary_grouped

comb_name  size_x  size_y    size
0          lh.EBA    4843    9879  7477.5
1        lh.FBA-1     748    1849  1380.5
2        lh.FBA-2     622    2769  1604.5
3        lh.FFA-1     790    1821  1108.5
4        lh.FFA-2     140    1593   973.0
..            ...     ...     ...     ...
63      rh.VWFA-2     465    2157  1209.0
64   rh.aTL-faces     100     710   434.0
65         rh.hMT     435     775   497.0
66         rh.hV4     785    1476  1049.5
67  rh.mTL-bodies     557    1009   667.0

[68 rows x 4 columns]

In [39]:
size_summary_file_LaTex = open(os.path.join(NSD_top_dir,
                                         'results',
                                         'Visual_ROI_sizes.txt'), 'w')
for curr_row in size_summary_grouped.iterrows():
    curr_line = '{} & {} \\\\'.format(curr_row[1]['comb_name'],
                                             curr_row[1]['size'],
                                           )
    #& {} & {} 
    # curr_row[1]['size_x'],curr_row[1]['size_y']
    size_summary_file_LaTex.write(curr_line)
size_summary_file_LaTex.close()



In [9]:
data_frame = pd.DataFrame(included_rois, )

['lh.V1d',
 'lh.V1v',
 'lh.V2d',
 'lh.V2v',
 'lh.V3d',
 'lh.V3v',
 'lh.hV4',
 'lh.OFA',
 'lh.FFA-1',
 'lh.FFA-2',
 'lh.OWFA',
 'lh.VWFA-1',
 'lh.VWFA-2',
 'lh.mfs-words',
 'lh.OPA',
 'lh.PPA',
 'lh.RSC',
 'lh.EBA',
 'lh.FBA-1',
 'lh.FBA-2',
 'lh.VO1',
 'lh.VO2',
 'lh.PHC1',
 'lh.PHC2',
 'lh.hMT',
 'lh.LO2',
 'lh.LO1',
 'lh.V3b',
 'lh.V3a',
 'lh.IPS0',
 'lh.IPS1',
 'lh.IPS2',
 'lh.IPS3',
 'rh.V1d',
 'rh.V1v',
 'rh.V2d',
 'rh.V2v',
 'rh.V3d',
 'rh.V3v',
 'rh.hV4',
 'rh.OFA',
 'rh.FFA-1',
 'rh.FFA-2',
 'rh.aTL-faces',
 'rh.OWFA',
 'rh.VWFA-1',
 'rh.VWFA-2',
 'rh.OPA',
 'rh.PPA',
 'rh.RSC',
 'rh.EBA',
 'rh.FBA-1',
 'rh.FBA-2',
 'rh.mTL-bodies',
 'rh.VO1',
 'rh.VO2',
 'rh.PHC1',
 'rh.PHC2',
 'rh.MST',
 'rh.hMT',
 'rh.LO2',
 'rh.LO1',
 'rh.V3b',
 'rh.V3a',
 'rh.IPS0',
 'rh.IPS1',
 'rh.IPS2',
 'rh.IPS3']

In [13]:
output_dir = os.path.join(NSD_top_dir,
                         'intermediate',
                         'masks',
                         'grouped_visual_ROIs')
#'/home/qilin1/NSD_GenPFC/intermediate/masks/grouped_visual_ROIs'
for curr_sub in sub_list:
    sub_roi_dict = {}
    for curr_hemi in hemi_list:
        # PRF-mapped ROIs
        for curr_roi_ind in prf_roi_list:
            curr_key = '{}.{}'.format(curr_hemi,
                                   prf_roi_name_list[curr_roi_ind-1])
            if curr_key in included_rois:
                sub_roi_dict[curr_key]=extract_prf_roi(curr_sub,
                                                      curr_roi_ind,
                                                      curr_hemi,
                                                      FS_top_dir)
            else:
                continue
        # Functional localizer ROIs
        for curr_roi_ind, curr_roi in enumerate(floc_rois):
            curr_key = '{}.{}'.format(curr_hemi,curr_roi)
            if curr_key in included_rois:
                sub_roi_dict[curr_key] = extract_floc_roi(curr_sub, 
                                                         curr_roi, 
                                                         domain_list[curr_roi_ind], 
                                                         curr_hemi, 
                                                         FS_top_dir,
                                                         probROIs_dir, vis_resp=True)
            else:
                continue
        
        # Kastner 2015 ROIs
        for curr_roi_ind, curr_roi_code in enumerate(kastner_rois):
            curr_key = '{}.{}'.format(curr_hemi,kastner_roi_names[curr_roi_ind])
            if curr_key in included_rois:
                sub_roi_dict[curr_key] = extract_Kaster_roi(curr_sub,
                                                            curr_roi_code,
                                                            curr_hemi,
                                                            FS_top_dir,
                                                            vis_resp=True)
            else:
                continue
    # Check if there's overlap in terms of the ROIs (i.e., the same vertex is assigned to different ROIs)
    lh_count = 0
    rh_count = 0
    for curr_key in sub_roi_dict.keys():
        if curr_key[:2]=='lh':
            if lh_count == 0:
                lh_count += 1
                lh_roi_array = sub_roi_dict[curr_key]
            else:
                lh_roi_array = lh_roi_array+sub_roi_dict[curr_key]
        elif curr_key[:2]=='rh':
            if rh_count == 0:
                rh_count += 1
                rh_roi_array = sub_roi_dict[curr_key]
            else:
                rh_roi_array = rh_roi_array+sub_roi_dict[curr_key]
        
    print(lh_roi_array[lh_roi_array>1])
    print(rh_roi_array[rh_roi_array>1])
    
    # Save the output
    sub_output_dir = os.path.join(output_dir, 'sub0{}.visualROIs'.format(curr_sub))
    pickle.dump(sub_roi_dict, open(sub_output_dir, 'wb'))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
